In [1]:
from google.cloud import bigquery
client = bigquery.Client()

query = ''' select DISTINCT ST_DOMAIN FROM TELCO_MDI.SMALL_MID_MARKET_RANDOM_SAMPLE where ST_DOMAIN IS NOT NULL LIMIT 50000'''
query_job = client.query(query,location = 'US')
df = query_job.to_dataframe()

In [ ]:
df.head()

In [5]:
#!/usr/bin/env python
# coding: utf-8

# ### Key word Crawler 
# 
# <b>Steps to follow :</b><br>
# 1. Upload input xlsx file having list of keywords, threads, input URL<br>
# 2. Please update the path of chromedriver, search for "chromedriver_path" in the code replace the existing path<br>
# 3. Please update the output path, where the results file has be stored<br>
# 4. After making the above changes Intialize/Run every cell in the sequence
# 

keyword_lst= [
'Appointment',
'Attention',
'Challenging Time',
'Closing',
'Company Update',
'Corona',
'Coronavirus',
'Covid',
'Critical',
'Curbside ',
'Delivery',
'Discount',
'Health',
'In-person',
'Layoff',
'Offering',
'Open',
'Outbreak',
'Pandemic',
'Safety',
'SARS-CoV-2',
'Virus',
]
# #### Importing all key packages

# In[113]:


import pandas as pd
import threading
import platform
import random
import re
import csv
import time
import PyPDF2
import requests

from bs4 import BeautifulSoup
from io import BytesIO
from fuzzywuzzy import fuzz
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from threading import Thread
import concurrent.futures as ft
import requests
import time
import ast
import nltk
import itertools

nltk.download('punkt')


# #### Code body 

# In[162]:



def url_check(input_url):
    url_low = input_url.lower()
    #url_low = url_low.split(';')[0]
    if not url_low.startswith('http'):
        url_low = 'http://' + url_low
    return url_low



## driversList would be a global var
## driverLock using lock capability of multithreading package ## checking in documentation

def getDriver(i,lock):
    global driversList
    global driverCnt
    driverReady = False
    driverMax = 20
    while not driverReady:
        driverReady = True
        with lock:
#             if len(driversList):
#                 tempD = driversList.pop()
#                 print("pop")
                
            if driverCnt < driverMax:

                chrome_options = webdriver.chrome.options.Options()
                chrome_options.add_argument('--dns-prefetch-disable')
                chrome_options.add_argument('--no-sandbox')
                chrome_options.add_argument('--lang=en-US')
                chrome_options.add_argument('--headless')
                chrome_options.add_argument('--disable-gpu')

                tempD = webdriver.Chrome(executable_path='D:/Telco MD&I/Code/chromedriver.exe', options=chrome_options)
                driverCnt += 1
                #driversList.append(tempD)
                #print(driverCnt)
                
            elif len(driversList):
                tempD = driversList.pop()
                #print("pop")
                ##
                #driversList
                
            else:
                driverReady = False
    return(tempD)



class CovidCrawler():

    display_name = "keyword Crawler"
    tag_name = "keyword_crawler"
    requires = []
    config = {}


    def key_word_match(self, text):
        # text = text.replace('\n',' ')
        text_len = len(text)
        text_lower = text.lower()

        is_output = False
        # output_dict = {}
        sent_text = nltk.sent_tokenize(text.replace("\n","").replace("\r","").replace("\t"," ")) 
        
        output_list = []
        cnt_sent = 0 
        for sent in sent_text:
            if any(x in sent.lower() for x in self.search_words):
                output_list.append(sent.replace("   ",""))
                cnt_sent = cnt_sent + 1
                
        
        
        for each_keyword in self.search_words :
            search = re.finditer(each_keyword.lower(),text_lower)
            cnt = 0
            matching = []
            for m in search:
                is_output = True
                cnt = cnt + 1

                start_idx = m.start()
                end_idx = m.end()

                start_idx = start_idx - self.win_size
                end_idx = end_idx + self.win_size

                if start_idx < 0:
                    start_idx = 0
                if end_idx > text_len-1:
                    end_idx = text_len-1

                matching.append(text[start_idx:end_idx].replace("\n","").replace("\r","").replace("   ",""))
            output_list.append(matching)
            
        if len(output_list) >=1:
            output_list = list(filter(None, output_list)) 
            cnt = len(output_list)

        return output_list, is_output, cnt

    def href_check_w_keywords(self):

        urls = []
        cnt_url= 0
        html_a = self.soup.findAll('a', href = True)
        for each_a in html_a:
            text = each_a.text.lower()
            href = each_a['href']
            if not href:
                continue

            for each_keyword in self.url_search_words:
                if each_keyword.lower() in text or each_keyword.lower() in href.lower():
                    urls.append(href)
                    cnt_url = cnt_url +1

        urls = list(set(urls))

        return urls,cnt_url

    def extract_info(self,i,url,lock):
        
        global driversList
        
        if str(url) !='nan':
            old_url= url
            url= url_check(url)

            if (i %1000 == 0):
                print(i,url)
                
               


            user_agent='Chrome/35.0.1916.47 Chrome/57.0.2987.110 Chrome/61.0.3163.91 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/537.36'
            headers = {'User-Agent': user_agent,'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

            self.search_words = main_search_words
            self.url_search_words = url_search_words
            self.win_size = main_page_char_win

            is_crawled = '1'
            #main page search
            try:
                req = requests.get(url.lower(),headers=headers,timeout=30)
                
                self.soup = BeautifulSoup(req.content, 'lxml')
            #time.sleep(1)
                self.body = self.soup.find('body')
                raw_text = self.body.get_text()
            except:
                try:
#                     chrome_options = webdriver.chrome.options.Options()
#                     chrome_options.add_argument('--dns-prefetch-disable')
#                     chrome_options.add_argument('--no-sandbox')
#                     chrome_options.add_argument('--lang=en-US')
#                     chrome_options.add_argument('--headless')
#                     chrome_options.add_argument('--disable-gpu')
#                     self.driver = webdriver.Chrome(executable_path=chromedriver_path, options=chrome_options)
                    self.driver = getDriver(i,lock)
                    self.driver.get(url.lower())
                    self.soup = BeautifulSoup(self.driver.page_source, 'lxml')
                    with lock:
                        driversList.append(self.driver) 
                        #** should be inside the lock
                    self.body = self.soup.find('body')
                    raw_text = self.body.get_text()
                    #self.driver.close()
                    req = ''
                except:
                    raw_text = ""
                    self.body = ""

            if (raw_text=='') or any(x in raw_text.lower() for x in ["blocked","bad url","(error 404)","403 error","doesn't exist (404)","404 not found","page not found","access denied","database error","this page isn't available"]):
                is_crawled = '0'
                output_text = [""]
                url_list = [""]
                output_list = ['']
                cnt_sent = ""
                cnt_url = ""


            else :
                try:
                    output_text = []
                    output_list, is_output, cnt_sent = self.key_word_match(self.body.get_text())
                    output_text.append(output_list)
                except:
                    output_text = [""]
                    output_list = [""]

                url_list =[""]
                # sub page search
#                 try:
#                     url_list = []
#                     covid_urls, cnt_url = self.href_check_w_keywords()
#                     url_list.append(covid_urls)
#                 except:
#                     url_list = [""]
           

#             temp.at[i,'output_text'] = str(output_list)
#             temp.at[i,'sub_url_list'] = str(url_list)
#             temp.at[i,'Resolve'] = is_crawled
#             temp.at[i,'# of sentences having keyword'] =cnt_sent
#             temp.at[i,'# of URLs having keyword']= cnt_url

            df= pd.DataFrame({'url':[str(old_url)],
                                 'output_text' : output_text,
                                 'sub_url_list': url_list,
                                 'Resolve':[is_crawled],
                                 '# of sentences having keyword':[cnt_sent],
                                 '# of URLs having keyword':[cnt_url]})
            if (i % 20000 == 0) and i != 0:
                print(round(time.perf_counter(),2))
                temp.to_csv("D:/Telco MD&I/Covid crawler/output_VZ_keyword_{}.csv".format(i))
            
        else:
            is_crawled = '0'
            output_text = [""]
            output_list = ""
            covid_urls = ""
            url_list = [""]
            cnt_sent = ""
            cnt_url = ""
            
#             temp.at[i,'output_text'] = output_list
#             temp.at[i,'sub_url_list'] = covid_urls
#             temp.at[i,'Resolve'] = is_crawled
#             temp.at[i,'# of sentences having keyword'] =cnt_sent
#             temp.at[i,'# of URLs having keyword']= cnt_url

            df= pd.DataFrame({'url':[str(old_url)],
                                 'output_text' : output_text,
                                 'sub_url_list': url_list,
                                 'Resolve':[is_crawled],
                                 '# of sentences having keyword':[cnt_sent],
                                 '# of URLs having keyword':[cnt_url]})
    
    return(str(df.values.to_list()))
# In[167]:


if __name__ == "__main__":
    # execute only if run as a script
    
    ### update input and output file paths 
    #input_excel_file = 'PLEASE PLACE INPUT FILE PATH'
    output_csv_file = 'D:/Telco MD&I/Covid crawler/output_VZ_keyword_1M.csv'
    chromedriver_path = 'D:/Telco MD&I/Code/chromedriver.exe'
    df = df
 
    
    #### Parameter initialziations
#     main_search_words = input_excel_file['Keywords']
#     url_search_words = input_excel_file['Keywords']
#     threads = input_excel_file['Threads']
    main_search_words = keyword_lst
    url_search_words = keyword_lst
    threads = 200
    main_page_char_win = 200
    google_search = False
    covid_crawler = CovidCrawler()
    start = time.perf_counter()
    temp = df[['ST_DOMAIN']] 
    concurrent = 500
    
    temp['output_text'] = 0
    temp['sub_url_list'] = 0
    temp['Resolve'] = 0
    temp['# of sentences having keyword'] =0
    temp['# of URLs having keyword']= 0
    
    lock =threading.Lock()
    driverCnt = 0
    driversList = []
    ##### MultiThreading code
    with ft.ThreadPoolExecutor(max_workers = threads) as executor:
        rt = executor.map(covid_crawler.extract_info,temp.index, temp['ST_DOMAIN'],itertools.repeat(lock))
    end = time.perf_counter()
    print(end-start)
#     lol = [ast.literal_eval(r) for r in rt]
#     df = [pd.DataFrame(s) for s in lol]
#     res = pd.concat(df)
#     res.columns  = ['url', 'output_text', 'sub_url_list', 'Resolve',
#        '# of sentences having keyword', '# of URLs having keyword']
#     res = res.reset_index(drop=True)
    temp.to_csv(output_csv_file)


# 1.	Update input_excel_file = ‘'D:/Telco MD&I/websites/Websites_15670.xlsx’
# 2.	Update output_excel_file = ‘'D:/Telco MD&I/Covid crawler/output.csv'’
# 3.	Keywords = [‘products’,’services’,’wireline’]
# 4.	Threads = 300 
# 5.	Update chromedriver_path = ‘D:/Telco MD&I/chromedriver.exe’


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\v.behl.snappoc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
covid_crawler.extract_info(0, temp['ST_DOMAIN'][0],lock)

0 http://rebeccaburrlaw.com


ValueError: invalid literal for int() with base 10: '[]'

In [3]:
temp.head()

ST_DOMAIN  output_text  sub_url_list  Resolve  \
0  rebeccaburrlaw.com            0             0        0   
1      ngklawfirm.com            0             0        0   
2  attorneystrust.com            0             0        0   
3     schevecklaw.com            0             0        0   
4         kmg-law.com            0             0        0   

   # of sentences having keyword  # of URLs having keyword  
0                              0                         0  
1                              0                         0  
2                              0                         0  
3                              0                         0  
4                              0                         0

In [ ]:
import threading
import pandas as pd
import platform
import random
import re
import csv
import time
import PyPDF2
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from fuzzywuzzy import fuzz
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from threading import Thread
import concurrent.futures as ft
import requests
import time
import ast
import nltk
import itertools

In [ ]:
def getDriver(i,lock):
    global driversList
    global driverCnt
    driverReady = False
    driverMax = 5
    while not driverReady:
        driverReady = True
        with lock:
            if len(driversList):
                tempD = driversList.pop()
            elif driverCnt < driverMax:
                print("creating new")
                #init driver
                chrome_options = webdriver.chrome.options.Options()
                chrome_options.add_argument('--dns-prefetch-disable')
                chrome_options.add_argument('--no-sandbox')
                chrome_options.add_argument('--lang=en-US')
                #chrome_options.add_argument('--headless')
                chrome_options.add_argument('--disable-gpu')

                tempD = webdriver.Chrome(executable_path='chromedriver.exe', options=chrome_options)
                driverCnt += 1
            else:
                driverReady = False
    return(tempD)

def crawler(i,url,lock):
    
    global driversList
    url_low = url.lower()
    #url_low = url_low.split(';')[0]
    if not url_low.startswith('http'):
        url_low = 'http://' + url_low

    user_agent='Chrome/35.0.1916.47 Chrome/57.0.2987.110 Chrome/61.0.3163.91 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/537.36'
    headers = {'User-Agent': user_agent,'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

    is_crawled = '1'

#     except:
    try:
#                     chrome_options = webdriver.chrome.options.Options()
#                     chrome_options.add_argument('--dns-prefetch-disable')
#                     chrome_options.add_argument('--no-sandbox')
#                     chrome_options.add_argument('--lang=en-US')
#                     chrome_options.add_argument('--headless')
#                     chrome_options.add_argument('--disable-gpu')
#                     self.driver = webdriver.Chrome(executable_path=chromedriver_path, options=chrome_options)
        start = time.perf_counter()
        driver = getDriver(i,lock)
        print("driver start length:",len(driversList))
        print("using for 3 seconds :",url)
        time.sleep(3)
        #driver.get(url_low.lower())
        #soup = BeautifulSoup(driver.page_source, 'lxml')
        driversList.append(driver)
        end = time.perf_counter()
        print( "{} driver end : {} {}".format(url,driver,round(start- end),2),)
        #body = soup.find('body')
        #raw_text = body.get_text()
        #print(url,":",len(raw_text))
        driver.close()
    except:
        raw_text = ""
        body = ""
    return(len(body))

    
    

In [ ]:
df = pd.read_csv("Dummy_data.csv")

In [ ]:
df= df[0:50]

In [ ]:
df

In [ ]:
lock =threading.Lock()
driverCnt = 0
driversList = []
with ft.ThreadPoolExecutor(max_workers = 10) as executor:
    executor.map(crawler,df.index, df.website,itertools.repeat(lock))
    #end = time.perf_counter()


In [ ]:
itertools.repeat(lock)

In [ ]:
df

In [ ]:
import itertools

In [ ]:
base = pd.read_csv('D:\Telco MD&I\Look_alike_model\Output\VZ_leads_MM_MB_COMBINED_3P.csv')

In [ ]:
base = base[base.segment == 'Mid-Market']

In [ ]:
base.columns

In [ ]:
base[base.Final_url.isna()==False]

In [ ]:
str(base['Final_url'][1]) == 'nan'

In [ ]:
x =[[], [], [], [], [], [], [], [], [], [], [], ['Learn moreGoogle AdsenseADVERTISEMENTADVERTISEMENTADVERTISEMENTCustomer ServiceContact UsOrder StatusGive Us']] 

In [ ]:
list(filter(None, x)) 

In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')

In [ ]:
nltk.download('punkt')